In [1]:
# importing libraries
import pandas as pd
import requests
import plotly.express as px
import streamlit as st

In [2]:
class ClimateDataset:
    """Base class for loading and processing climate-related datasets."""

    def __init__(self, name):
        self.name = name
        self.data = pd.DataFrame()

    def load_data(self):
        raise NotImplementedError("Subclasses must implement this method.")

    def clean_data(self):
        if not self.data.empty:
            self.data.dropna(inplace=True)
        return self.data

    def summarize(self):
        return self.data.describe()

In [3]:
class WorldBankDataset(ClimateDataset):
    """Fetches climate indicators from the World Bank Climate Data API."""

    BASE_URL = "https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json&per_page=1000"

    INDICATOR_NAMES = {
        'EN.ATM.CO2E.PC': 'CO₂ Emissions (metric tons per capita)',
        'EN.ATM.CO2E.KT': 'CO₂ Emissions (kt)',
        'AG.LND.FRST.ZS': 'Forest Area (% of land area)',
        'EN.ATM.PM25.MC.M3': 'PM2.5 Air Pollution (µg/m³)',
        'EN.ATM.GHGT.KT.CE': 'Total Greenhouse Gas Emissions (kt of CO₂ equivalent)'
    }

    def __init__(self, country='WLD', indicator='EN.ATM.CO2E.PC'):
        name = f"World Bank: {self.INDICATOR_NAMES.get(indicator, indicator)}"
        super().__init__(name)
        self.country = country
        self.indicator = indicator

    def load_data(self):
        url = self.BASE_URL.format(country=self.country, indicator=self.indicator)
        response = requests.get(url)
        
        if response.status_code != 200:
            raise ConnectionError(f"Failed to fetch data: {response.status_code}")

        json_data = response.json()

        if len(json_data) < 2:
            raise ValueError("Unexpected World Bank API response format.")

        records = json_data[1]
        df = pd.DataFrame(records)
        self.data = df[['date', 'value']].rename(columns={'date': 'year', 'value': 'indicator_value'})
        self.data['year'] = pd.to_numeric(self.data['year'], errors='coerce')
        self.data.dropna(inplace=True)
        self.data.sort_values('year', inplace=True)
        return self.data

In [4]:
class ClimateVisualizer:
    """Manages visualization and interaction for climate datasets."""

    def __init__(self, dataset):
        self.dataset = dataset

    def plot(self, start_year=None, end_year=None):
        df = self.dataset.data
        if df.empty:
            raise ValueError("Dataset is empty. Load data first.")

        if start_year and end_year:
            df = df[(df['year'] >= start_year) & (df['year'] <= end_year)]

        fig = px.line(df, x='year', y='indicator_value', title=f"{self.dataset.name} Over Time")
        return fig

## Dashboard

In [5]:
def run_dashboard():
    st.title("🌍 Climate Change Data Visualizer")
    st.sidebar.header("Configuration")

    country = st.sidebar.text_input("Enter country code (e.g., WLD, KE, UG):", value='WLD')

    indicator_options = {
        'CO₂ Emissions (metric tons per capita)': 'EN.ATM.CO2E.PC',
        'CO₂ Emissions (kt)': 'EN.ATM.CO2E.KT',
        'Forest Area (% of land area)': 'AG.LND.FRST.ZS',
        'PM2.5 Air Pollution (µg/m³)': 'EN.ATM.PM25.MC.M3',
        'Total Greenhouse Gas Emissions (kt CO₂e)': 'EN.ATM.GHGT.KT.CE'
    }

    indicator_label = st.sidebar.selectbox(
        "Select indicator:", list(indicator_options.keys())
    )
    indicator = indicator_options[indicator_label]

    dataset = WorldBankDataset(country=country, indicator=indicator)

    try:
        with st.spinner('Fetching data from World Bank API...'):
            df = dataset.load_data()
            dataset.clean_data()
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return

    st.success(f"Loaded {len(dataset.data)} records for {indicator_label}.")

    # Date range selector
    min_year, max_year = int(df['year'].min()), int(df['year'].max())
    start_year, end_year = st.slider(
        'Select year range:',
        min_value=min_year, max_value=max_year,
        value=(min_year, max_year)
    )

    visualizer = ClimateVisualizer(dataset)
    fig = visualizer.plot(start_year=start_year, end_year=end_year)
    st.plotly_chart(fig, use_container_width=True)

    st.write("### Data Preview")
    st.dataframe(df)

In [6]:
if __name__ == "__main__":
    run_dashboard()

2025-11-07 17:56:51.880 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-07 17:56:52.166 
  command:

    streamlit run /Users/jonathan/Documents/MSc Data Sci/Project Repositories/climate-change-data-visualiser/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-07 17:56:52.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-07 17:56:52.168 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-07 17:56:52.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-07 17:56:52.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-07 17:56:52.171 Thread 'MainThread': missing ScriptRunContext! This warning can be